# What happens if I save the same artifacts & records twice?

LaminDB's operations are idempotent in the sense defined in this document.

This allows you to re-run a notebook or script without erroring or duplicating data. Similar behavior holds for human data entry.

## Summary

### Metadata records

If you try to create any metadata record ({class}`~lamindb.core.Record`) and {attr}`~lamindb.core.subsettings.CreationSettings.search_names` is `True` (the default): 

1. LaminDB will warn you if a record with similar `name` exists and display a table of similar existing records.
2. You can then decide whether you'd like to save a record to the database or rather query an existing one from the table.
3. If a name already has an exact match in a registry, LaminDB will return it instead of creating a new record. For versioned entities, also the version must be passed.

If you set {attr}`~lamindb.core.subsettings.CreationSettings.search_names` to `False`, you'll directly populate the DB.

### Data: artifacts & collections

If you try to create a {class}`~lamindb.Artifact` object from the same content, depending on {attr}`~lamindb.core.subsettings.CreationSettings.artifact_if_hash_exists`, 

- you'll get an existing object, if `creation.artifact_if_hash_exists = "warn_return_existing"` (the default)
- you'll get an error, if `creation.artifact_if_hash_exists = "error"`
- you'll get a warning and a new object, if `creation.artifact_if_hash_exists = "warn_create_new"`

## Examples

In [ ]:
# !pip install 'lamindb[jupyter]'
!lamin init --storage ./test-idempotency

In [ ]:
import lamindb as ln
import pytest

ln.settings.verbosity = "hint"
ln.settings.transform.stem_uid = "ANW20Fr4eZgM"
ln.settings.transform.version = "1"
ln.context.track()

### Metadata records

In [ ]:
assert ln.settings.creation.search_names

Let us add a first record to the {class}`~lamindb.ULabel` registry:

In [ ]:
label = ln.ULabel(name="My project 1")
label.save()

If we create a new record, we'll automatically get search results that give clues on whether we are prone to duplicating an entry:

In [ ]:
label = ln.ULabel(name="My project 1a")

In [ ]:
label.save()

In case we match an existing name directly, we'll get the existing object:

In [ ]:
label = ln.ULabel(name="My project 1")

If we save it again, it will not create a new entry in the registry:

In [ ]:
label.save()

Now, if we create a third record, we'll get two alternatives:

In [ ]:
label = ln.ULabel(name="My project 1b")

If we prefer to not perform a search, e.g. for performance reasons or too noisy logging, we can switch it off.

In [ ]:
ln.settings.creation.search_names = False

In [ ]:
label = ln.ULabel(name="My project 1c")

In this walkthrough, switch it back on:

In [ ]:
ln.settings.creation.search_names = True

### Data: artifacts and collections

#### Warn upon trying to re-ingest an existing artifact

In [ ]:
assert ln.settings.creation.artifact_if_hash_exists == "warn_return_existing"

In [ ]:
filepath = ln.core.datasets.file_fcs()

Create an `Artifact`:

In [ ]:
artifact = ln.Artifact(filepath, description="My fcs artifact")
artifact.save()

In [ ]:
assert artifact.hash == "KCEXRahJ-Ui9Y6nksQ8z1A"

Create an `Artifact` from the same path:

In [ ]:
artifact2 = ln.Artifact(filepath, description="My fcs artifact")

It gives us the existing object:

In [ ]:
assert artifact.id == artifact2.id
assert artifact.run == artifact2.run

If you save it again, nothing will happen (the operation is idempotent):

In [ ]:
artifact2.save()

In the hidden cell below, you'll see how this interplays with data lineage.

In [ ]:
ln.context.track(new_run=True)
artifact3 = ln.Artifact(filepath, description="My fcs artifact")
assert artifact3.id == artifact2.id
assert artifact3.run != artifact2.run
assert artifact3._previous_runs.first() == artifact2.run

#### Error upon trying to re-ingest an existing artifact

In [ ]:
ln.settings.creation.artifact_if_hash_exists = "error"

In this case, you'll not be able to create an object from the same content:

In [ ]:
with pytest.raises(FileExistsError):
    artifact3 = ln.Artifact(filepath, description="My new fcs artifact")

#### Warn and create a new artifact

Lastly, let us discuss the following setting:

In [ ]:
ln.settings.creation.artifact_if_hash_exists = "warn_create_new"

In this case, you'll create a new object:

In [ ]:
artifact4 = ln.Artifact(filepath, description="My new fcs artifact")
artifact4.save()

You can verify that it's a new entry by comparing the ids:

In [ ]:
assert artifact4.id != artifact.id

In [ ]:
ln.Artifact.filter(hash="KCEXRahJ-Ui9Y6nksQ8z1A").df()

In [ ]:
assert len(ln.Artifact.filter(hash="KCEXRahJ-Ui9Y6nksQ8z1A").all()) == 2

In [ ]:
!lamin delete --force test-idempotency